In [1]:
import pandas as pd
from google.colab import files
from google.colab import drive
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch

In [2]:
# Open a file upload dialog
# Select here all files to upload!
# If already uploaded, just press 'Cancel Upload'
# Upload the original english comments, but labelled by the majority vote
uploaded = files.upload()

Saving High_Confidence_Comments_Original_English.csv to High_Confidence_Comments_Original_English.csv


In [3]:
# Set the path to the data
# On local machine use the relative path, for example
# path = 'NLP labelled data preview/english set/'
# On Google Colab use this path
# '/content/'
path = '/content/'

In [4]:
# Load the dataset
file = 'High_Confidence_Comments_Original_English.csv'
all_comments = pd.read_csv(path + file)


In [5]:
##### TESTING PURPOSES #####
# Just take the first 10 comments

all_comments = all_comments.loc[:10, :]

In [6]:
# Load the translation models
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [7]:
# Try to use GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_

In [11]:
def translate_text(text, source_lang, target_lang):
    tokenizer.src_lang = source_lang
    encoded_text = tokenizer(text, return_tensors="pt").to(device)
    generated_tokens = model.generate(
        **encoded_text,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang]
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]


In [9]:
comments_column = "Comment" # The column with the comments
source_lang =  "en_XX"  # Source language (always english)
french_lang =  "fr_XX"
german_lang = "de_DE"
spanish_lang = "es_XX"
italian_lang = "it_IT"

In [12]:
# Translate in all languages
all_comments["translated_french"] = all_comments[comments_column].apply(lambda x: translate_text(x, source_lang, french_lang))
all_comments["translated_german"] = all_comments[comments_column].apply(lambda x: translate_text(x, source_lang, german_lang))
all_comments["translated_spanish"] = all_comments[comments_column].apply(lambda x: translate_text(x, source_lang, spanish_lang))
all_comments["translated_italian"] = all_comments[comments_column].apply(lambda x: translate_text(x, source_lang, italian_lang))


In [13]:
# Save to csv file
all_comments.to_csv(path + 'High_Confidence_Comments_Translated.csv', index=False)

In [14]:
# Download the file to your local machine (from google colab)
files.download(path + 'High_Confidence_Comments_Translated.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>